In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

## Link

In [19]:
start_date = '2022-09-01'
end_date = '2022-09-30'

In [20]:
link = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(link_lifetime_last_ride_date) 
    link_lifetime_last_ride_date,
    link_lifetime_first_ride_date,
    --MAX(link_lifetime_rides) 
    link_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (link_lifetime_rides > 0 or link_lifetime_rides is not null or link_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN link_lifetime_rides IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 THEN '1-4'
    WHEN (link_lifetime_rides BETWEEN 5 AND 20) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '5-20'
    WHEN (link_lifetime_rides BETWEEN 21 AND 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '21-50'
    WHEN (link_lifetime_rides > 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '50+'
    WHEN link_lifetime_rides >= 5 THEN '>=5'
    ELSE 'LTR=0'
    END ltr_link,
    CASE 
    WHEN link_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '1. LAST_30_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '2. 31_90_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '3. 91_180_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '4. 181_365_DAYS'
    WHEN link_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '5. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_link
FROM base_month_customer
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Link'
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_link,'NEW') ltr_link,
        COALESCE(b.recency_link, 'NEW') recency_link,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_link,
        case 
        when ltr_link = '1-4' then 1
        when ltr_link = '5-20' then 2
        when ltr_link = '21-50' then 3
        when ltr_link = '50+' then 4
        when ltr_link = '>=5' then 5
        when ltr_link = 'LTR=0' then 6
        when ltr_link = 'NEW' then 7
        end row_order,
        recency_link,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_link,
        a.recency_link,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_link,
        '' recency_link,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_link,recency_link,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3

"""

In [21]:
df_link = pd.read_sql(link, conn)

In [22]:
df_link_sep = df_link
df_link_sep.head()

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-09-01,1-4,1. LAST_30_DAYS,1642301,6.09,1047277,63.77,925256,56.34,601090,...,29.23,79.86,10.52,1202587,8.00,2.51,5222698.0,73278959.0,7.127145,4.342886
1,2022-09-01,1-4,2. 31_90_DAYS,2127715,7.90,864891,40.65,719026,33.79,366917,...,12.35,71.60,5.76,482735,3.21,1.84,2208773.0,31269240.0,7.063725,4.575539
2,2022-09-01,1-4,3. 91_180_DAYS,2072596,7.69,554664,26.76,452855,21.85,205607,...,6.88,69.39,3.13,247187,1.64,1.73,1249967.0,16264962.0,7.685029,5.056767
3,2022-09-01,1-4,4. 181_365_DAYS,1931183,7.17,305808,15.84,254025,13.15,114891,...,4.07,68.37,1.72,138769,0.92,1.77,731182.0,9241566.0,7.911884,5.269059
4,2022-09-01,1-4,5. 365_ABOVE,5379504,19.96,329247,6.12,278097,5.17,128935,...,1.67,69.52,1.96,168323,1.12,1.88,924223.0,11074306.0,8.345652,5.490771


In [6]:
df_link_nov

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-11-01,1-4,1. LAST_30_DAYS,1626810,5.51,995351,61.18,874207,53.74,564284,...,27.34,78.81,9.39,1109346,7.01,2.49,4068936.0,67603337.0,6.018839,3.667869
1,2022-11-01,1-4,2. 31_90_DAYS,2179150,7.38,895202,41.08,747989,34.32,386166,...,12.61,71.14,5.80,509782,3.22,1.86,1976467.0,32533742.0,6.075130,3.877083
2,2022-11-01,1-4,3. 91_180_DAYS,2431872,8.24,644844,26.52,525925,21.63,238860,...,6.72,68.44,3.45,285568,1.80,1.75,1185069.0,18551946.0,6.387842,4.149866
3,2022-11-01,1-4,4. 181_365_DAYS,2457042,8.33,396332,16.13,325644,13.25,142020,...,3.91,67.68,2.03,166831,1.05,1.74,757076.0,10900794.0,6.945145,4.537982
4,2022-11-01,1-4,5. 365_ABOVE,5624523,19.06,330354,5.87,273938,4.87,126984,...,1.54,68.39,1.83,159592,1.01,1.84,781669.0,10457284.0,7.474876,4.897921
5,2022-11-01,5-20,1. LAST_30_DAYS,1664552,5.64,1318511,79.21,1237035,74.32,974575,...,50.62,86.46,17.80,2894488,18.29,3.44,8051290.0,167132592.0,4.817307,2.781594
6,2022-11-01,21-50,1. LAST_30_DAYS,764409,2.59,663818,86.84,638351,83.51,556229,...,66.35,91.18,10.71,2585594,16.33,5.10,7500019.0,142460075.0,5.264646,2.900695
7,2022-11-01,50+,1. LAST_30_DAYS,557823,1.89,510730,91.56,498465,89.36,461385,...,78.22,94.57,9.22,4171131,26.35,9.56,15509721.0,220412976.0,7.036664,3.718349
8,2022-11-01,>=5,2. 31_90_DAYS,2023875,6.86,1159941,57.31,1036667,51.22,678742,...,26.61,79.34,11.38,1234319,7.80,2.29,3527919.0,74484671.0,4.736436,2.858191
9,2022-11-01,>=5,3. 91_180_DAYS,1478038,5.01,544903,36.87,466152,31.54,262373,...,13.26,74.72,4.14,390181,2.47,1.99,1171660.0,24145067.0,4.852585,3.002863


In [18]:
df_link_oct

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,1-4,1. LAST_30_DAYS,1651497,5.85,1062265,64.32,948541,57.44,613671,...,28.65,77.11,10.13,1108204,7.74,2.34,4190628.0,69015507.0,6.072009,3.781459
1,2022-10-01,1-4,2. 31_90_DAYS,2109155,7.47,888755,42.14,753157,35.71,381930,...,12.37,68.34,5.59,455101,3.18,1.74,1823711.0,30073885.0,6.064102,4.007267
2,2022-10-01,1-4,3. 91_180_DAYS,2303551,8.16,633867,27.52,527108,22.88,235515,...,6.67,65.28,3.29,252478,1.76,1.64,1096692.0,16920140.0,6.481578,4.343713
3,2022-10-01,1-4,4. 181_365_DAYS,2164498,7.67,358934,16.58,301305,13.92,131967,...,3.93,64.40,1.82,141003,0.99,1.66,661182.0,9525445.0,6.941219,4.689134
4,2022-10-01,1-4,5. 365_ABOVE,5492237,19.46,339452,6.18,287273,5.23,130760,...,1.55,65.26,1.83,149774,1.05,1.76,737190.0,9970166.0,7.393959,4.922016
5,2022-10-01,5-20,1. LAST_30_DAYS,1695321,6.01,1364740,80.50,1293819,76.32,1016082,...,51.08,85.23,18.54,2772698,19.37,3.20,7441086.0,164152361.0,4.533036,2.683699
6,2022-10-01,21-50,1. LAST_30_DAYS,758614,2.69,664039,87.53,642657,84.71,557719,...,66.26,90.12,10.76,2356014,16.46,4.69,6753901.0,133039298.0,5.076621,2.866664
7,2022-10-01,50+,1. LAST_30_DAYS,536110,1.90,492577,91.88,482650,90.03,444986,...,77.85,93.79,8.93,3574932,24.98,8.57,13245109.0,194141763.0,6.822390,3.704996
8,2022-10-01,>=5,2. 31_90_DAYS,1876918,6.65,1083000,57.70,976628,52.03,625411,...,25.32,76.00,10.17,994066,6.95,2.09,2744251.0,61932647.0,4.431025,2.760633
9,2022-10-01,>=5,3. 91_180_DAYS,1349466,4.78,513215,38.03,445795,33.03,246566,...,13.04,71.39,3.77,324661,2.27,1.84,963455.0,20680265.0,4.658814,2.967572


In [23]:
df_link_sep

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-09-01,1-4,1. LAST_30_DAYS,1642301,6.09,1047277,63.77,925256,56.34,601090,...,29.23,79.86,10.52,1202587,8.00,2.51,5222698.0,73278959.0,7.127145,4.342886
1,2022-09-01,1-4,2. 31_90_DAYS,2127715,7.90,864891,40.65,719026,33.79,366917,...,12.35,71.60,5.76,482735,3.21,1.84,2208773.0,31269240.0,7.063725,4.575539
2,2022-09-01,1-4,3. 91_180_DAYS,2072596,7.69,554664,26.76,452855,21.85,205607,...,6.88,69.39,3.13,247187,1.64,1.73,1249967.0,16264962.0,7.685029,5.056767
3,2022-09-01,1-4,4. 181_365_DAYS,1931183,7.17,305808,15.84,254025,13.15,114891,...,4.07,68.37,1.72,138769,0.92,1.77,731182.0,9241566.0,7.911884,5.269059
4,2022-09-01,1-4,5. 365_ABOVE,5379504,19.96,329247,6.12,278097,5.17,128935,...,1.67,69.52,1.96,168323,1.12,1.88,924223.0,11074306.0,8.345652,5.490771
5,2022-09-01,5-20,1. LAST_30_DAYS,1645938,6.11,1316550,79.99,1237858,75.21,971870,...,51.37,87.00,18.52,2940730,19.55,3.48,10268410.0,170180051.0,6.033851,3.491789
6,2022-09-01,21-50,1. LAST_30_DAYS,721276,2.68,628782,87.18,605129,83.90,526172,...,66.72,91.46,10.54,2487988,16.54,5.17,9193251.0,137760145.0,6.673375,3.695054
7,2022-09-01,50+,1. LAST_30_DAYS,498325,1.85,456247,91.56,445159,89.33,410932,...,77.95,94.53,8.51,3610905,24.01,9.30,15241693.0,193936982.0,7.859096,4.221017
8,2022-09-01,>=5,2. 31_90_DAYS,1810699,6.72,1020455,56.36,905189,49.99,581365,...,25.26,78.68,10.02,1014459,6.74,2.22,3678153.0,62438856.0,5.890808,3.625729
9,2022-09-01,>=5,3. 91_180_DAYS,1192290,4.42,443236,37.18,379312,31.81,211133,...,13.20,74.53,3.45,307853,2.05,1.96,1164385.0,19232267.0,6.054330,3.782276


In [24]:
frames = [df_link_nov, df_link_oct, df_link_sep]
nov_oct_sep = pd.concat(frames)
nov_oct_sep.to_csv("link_nov_oct_sep.csv", index = False)

In [25]:
nov_oct_sep

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-11-01,1-4,1. LAST_30_DAYS,1626810,5.51,995351,61.18,874207,53.74,564284,...,27.34,78.81,9.39,1109346,7.01,2.49,4068936.0,67603337.0,6.018839,3.667869
1,2022-11-01,1-4,2. 31_90_DAYS,2179150,7.38,895202,41.08,747989,34.32,386166,...,12.61,71.14,5.80,509782,3.22,1.86,1976467.0,32533742.0,6.075130,3.877083
2,2022-11-01,1-4,3. 91_180_DAYS,2431872,8.24,644844,26.52,525925,21.63,238860,...,6.72,68.44,3.45,285568,1.80,1.75,1185069.0,18551946.0,6.387842,4.149866
3,2022-11-01,1-4,4. 181_365_DAYS,2457042,8.33,396332,16.13,325644,13.25,142020,...,3.91,67.68,2.03,166831,1.05,1.74,757076.0,10900794.0,6.945145,4.537982
4,2022-11-01,1-4,5. 365_ABOVE,5624523,19.06,330354,5.87,273938,4.87,126984,...,1.54,68.39,1.83,159592,1.01,1.84,781669.0,10457284.0,7.474876,4.897921
5,2022-11-01,5-20,1. LAST_30_DAYS,1664552,5.64,1318511,79.21,1237035,74.32,974575,...,50.62,86.46,17.80,2894488,18.29,3.44,8051290.0,167132592.0,4.817307,2.781594
6,2022-11-01,21-50,1. LAST_30_DAYS,764409,2.59,663818,86.84,638351,83.51,556229,...,66.35,91.18,10.71,2585594,16.33,5.10,7500019.0,142460075.0,5.264646,2.900695
7,2022-11-01,50+,1. LAST_30_DAYS,557823,1.89,510730,91.56,498465,89.36,461385,...,78.22,94.57,9.22,4171131,26.35,9.56,15509721.0,220412976.0,7.036664,3.718349
8,2022-11-01,>=5,2. 31_90_DAYS,2023875,6.86,1159941,57.31,1036667,51.22,678742,...,26.61,79.34,11.38,1234319,7.80,2.29,3527919.0,74484671.0,4.736436,2.858191
9,2022-11-01,>=5,3. 91_180_DAYS,1478038,5.01,544903,36.87,466152,31.54,262373,...,13.26,74.72,4.14,390181,2.47,1.99,1171660.0,24145067.0,4.852585,3.002863
